## 只使用父节点课程预测成绩
形式语言与编译
### 父节点课程
* 操作系统原理i
* 操作系统设计专题实验
* 数据库系统
* 编译器设计专题实验
* 计算机系统综合设计实验
* 计算机组成
* 计算机组成与结构专题实验
* 计算机网络专题实验
* 计算机网络原理
* 软件定义网络


In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import median_absolute_error, mean_squared_error, r2_score, median_absolute_error

import scipy

data = pd.read_csv('cs15_22_parentsOnly_Compilation.csv')

#划分训练-测试集，测试集比例为0.25
X_train, X_test, y_train, y_test = train_test_split(data, data['形式语言与编译'], test_size = 0.25, random_state=42)

In [2]:
X_train.head()

,形式语言与编译,操作系统原理i,操作系统设计专题实验,数据库系统,编译器设计专题实验,计算机系统综合设计实验,计算机组成,计算机组成与结构专题实验,计算机网络专题实验,计算机网络原理,软件定义网络
974,66,79,92,68,85,88,79,79,76,82,89
620,0,0,0,0,0,0,0,0,0,0,0
777,60,71,92,62,77,83,63,83,98,69,74
427,78,92,92,86,90,76,85,85,92,96,0
199,80,60,92,76,77,87,76,95,87,69,82


In [3]:
import numpy as np
# Calculate mae and rmse
def evaluate_predictions(predictions, true):
    mae = np.mean(abs(predictions - true))
    rmse = np.sqrt(np.mean((predictions - true) ** 2))

    return mae, rmse

In [4]:
median_pred = X_train['形式语言与编译'].median()

median_preds = [median_pred for _ in range(len(X_test))]

true = X_test['形式语言与编译']

In [5]:
# Display the naive baseline metrics
mb_mae, mb_rmse = evaluate_predictions(median_preds, true)
print('Median Baseline  MAE: {:.4f}'.format(mb_mae))
print('Median Baseline RMSE: {:.4f}'.format(mb_rmse))

Median Baseline  MAE: 29.6994
Median Baseline RMSE: 40.6660


In [6]:
def calculate_rmsea(y_test, predictions):
    # Calculate the root mean square error of approximation (RMSEA)
    residuals = predictions - y_test
    chi_squared = np.sum(residuals ** 2)  # Chi-squared statistic
    degrees_of_freedom = len(y_test)  # Degrees of freedom
    n = len(y_test)  # Number of data points
    rmsea = np.sqrt(chi_squared / (degrees_of_freedom * (n - 1)))
    return rmsea

def evaluate(X_train, X_test, y_train, y_test):
    # Names of models
    model_name_list = ['Linear Regression', 'ElasticNet Regression',
                      'Random Forest', 'Extra Trees', 'SVM',
                       'Gradient Boosted', 'Baseline']
    X_train = X_train.drop('形式语言与编译', axis='columns')
    X_test = X_test.drop('形式语言与编译', axis='columns')
    
    # Instantiate the models
    model1 = LinearRegression()
    model2 = ElasticNet(alpha=1.0, l1_ratio=0.5)
    model3 = RandomForestRegressor(n_estimators=100)
    model4 = ExtraTreesRegressor(n_estimators=100)
    model5 = SVR(kernel='rbf', degree=3, C=1.0, gamma='auto')
    model6 = GradientBoostingRegressor(n_estimators=50)
    
    # Dataframe for results
    results = pd.DataFrame(columns=['mae', 'rmse', 'r2', 'rmsea'], index=model_name_list)

    # Train and predict with each model
    for i, model in enumerate([model1, model2, model3, model4, model5, model6]):
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        
        # Metrics
        mae = median_absolute_error(y_test, predictions)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        r2 = r2_score(y_test, predictions)
        rmsea = calculate_rmsea(y_test, predictions)
        # Insert results into the dataframe
        model_name = model_name_list[i]
        results.loc[model_name, :] = [mae, rmse, r2, rmsea]
    
    # Median Value Baseline Metrics
    baseline = np.median(y_train)
    baseline_mae = np.mean(abs(baseline - y_test))
    baseline_rmse = np.sqrt(np.mean((baseline - y_test) ** 2))
    baseline_residuals = y_test - baseline
    baseline_sse = np.sum(baseline_residuals ** 2)
    y_mean = np.mean(y_test)
    y_diff = y_test - y_mean
    sst = np.sum(y_diff ** 2)
    baseline_r2 = 1 - (baseline_sse / sst)
    baseline_rmsea = calculate_rmsea(y_test, baseline)
    results.loc['Baseline', :] = [baseline_mae, baseline_rmse, baseline_r2, baseline_rmsea]
    
    return results

In [7]:
results = evaluate(X_train, X_test, y_train, y_test)
print(results)

                             mae       rmse        r2     rmsea
Linear Regression       3.336223  12.780562  0.875004  0.678322
ElasticNet Regression   3.251391  12.759404  0.875418  0.677199
Random Forest              3.795  12.500832  0.880416  0.663475
Extra Trees                3.155  12.188661  0.886314  0.646907
SVM                     5.560319  17.596141  0.763064  0.933906
Gradient Boosted        3.251945  12.618701   0.87815  0.669731
Baseline               29.699438  40.666049 -0.265494   2.15833
